In [74]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os 
import ast
import json

https://stackoverflow.com/questions/62335537/scraping-text-from-kickstarter-projects-return-nothing

In [154]:
base = "/Users/ivankoh/Downloads/kickstarter data/"
filepaths = [base + f for f in os.listdir(base) if f.endswith('.csv')]
df = pd.concat(map(pd.read_csv, filepaths))
df_1 = df.drop_duplicates(subset=['id'])
df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])
print(len(df))
print(len(df_1))

392244
203510


/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_74056/4110031066.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_1['main_url'] = df_1['urls'].apply(lambda x: eval(x)['web']['project'])


In [157]:
def get_slug(url):
    return re.search('/projects/(.*)\?', url).group(1)

slugs = list(map(get_slug, df_1['main_url'].tolist()))
story_list = []
risk_list = []

s = requests.Session()
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:50.0) Gecko/20100101 Firefox/50.0'}
r = s.get("https://www.kickstarter.com", headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')
xcsrf = soup.find("meta", {"name": "csrf-token"})["content"]

query = """
query Campaign($slug: String!) {
  project(slug: $slug) {
    risks
    story(assetWidth: 680)
  }
}"""

for slug in slugs:
    print(f"--------{slug}------")
    r = s.post("https://www.kickstarter.com/graph",
        headers= {
            "x-csrf-token": xcsrf,
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'
        },
        json = {
            "operationName":"Campaign",
            "variables":{
                "slug": slug
            },
            "query": query
        })
    try:
        result = r.json()
        print("-------STORY--------")
        story_html = result["data"]["project"]["story"]
        soup = BeautifulSoup(story_html, 'html.parser')
        curr_story = []
        for i in soup.find_all('p'):
            print(i.get_text())
            curr_story.append(i.get_text())
        story_list.append(" ".join(curr_story) if curr_story else "")

        print("-------RISKS--------")
        print(result["data"]["project"]["risks"])
        risk_list.append(result["data"]["project"]["risks"] if result["data"]["project"]["risks"] else "")
    except:
        print(r)
        story_list.append("")
        risk_list.append("")
        continue

TypeError: 'NoneType' object is not subscriptable